# Neural Networks - MultiLayerPerceptron

# Wine Dataset
## Wines are categorized into 3 customer segments based on featuers listed below:
### Features are:

<ol>
    <li>Alcohol</li>
    <li>Malic acid</li>
    <li>Ash</li>
    <li>Alcalinity of ash</li>
    <li>Magnesium</li>
    <li>Total phenols</li>
    <li>Flavanoids</li>
    <li>Nonflavanoid phenols</li>
    <li>Proanthocyanins</li>
    <li>Color intensity</li>
    <li>Hue</li>
    <li>OD280/OD315 of diluted wines</li>
    <li>Proline </li>
    </ol>
</font>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, VectorIndexer

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("Wine Dataset MLP").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("Error")

In [ ]:
sdf = spark.read.format('csv').options(header='true', inferSchema='true').load('../datasets/Wine.csv')

In [ ]:
sdf.printSchema()

In [ ]:
sdf.show(3)

<font color = 'tomato'>
<h3>Data preparation</h3>
1. Convert the Customer_Segment (string) to label (number) <br>
2. Create features using Vector Assembler<br>
3. Create training and test set <br>
</font>

In [ ]:
# Convert the target column to numbers
labelIndexer = StringIndexer(inputCol="Customer_Segment", outputCol="label").fit(sdf)

In [ ]:
df = labelIndexer.transform(sdf)

In [ ]:
df.show(3)

In [ ]:
# Take all columns except Customer_Segment and label
fcols = df.columns[:-2]
fcols

In [ ]:
# Select features into vector assembler
from pyspark.ml.feature import VectorAssembler
vassemb = VectorAssembler(inputCols = fcols, outputCol = 'features')
ndf = vassemb.transform(df)
ndf = ndf.select(['label', 'features'])

ndf.show(3, truncate = False)
#ndf.printSchema()

### Apply StandardScaler

In [ ]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="stdFeatures",
                        withStd=False, withMean=True)

In [ ]:
scalerModel = scaler.fit(ndf)

In [ ]:
scaledData = scalerModel.transform(ndf)

In [ ]:
# scaledData.select("stdFeatures").show(3, truncate = False)

In [ ]:
(trainingData, testData) = scaledData.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

<font color = 'tomato'>
    <h3>Use MultiLayerPerceptron Classifier </h3>
    <ol>
        <li>Create MultiLayerPerceptronClassifier instance</li>
        <li>Make sure to define layers</li>
        <li>Fit the transformed features</li>
        <li>Transform the model</li>
        <li>Evaluate the model using multi-class classification evaluator</li>
        <li>Print Test Data accuracy</li>
        <li>Create Confusion Matrix</li>
    </ol>
</font>

## MultiLayerPerceptron Classifier

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
# There are 13 features and 3 Classes - Customer Segments)
layers = [13, 9, 7, 3]

In [ ]:
trainer = MultilayerPerceptronClassifier(featuresCol = 'stdFeatures', labelCol = 'label',
                                         maxIter = 100, layers = layers, blockSize = 3,
                                         seed = 100)

In [ ]:
model = trainer.fit(trainingData)

In [ ]:
result = model.transform(testData)

In [ ]:
predictionAndLabels = result.select("prediction", "label")

In [ ]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [ ]:
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

In [ ]:
result.select('label', 'prediction', 'probability', 'features').show(52)

In [ ]:
y_true = result.select('label')
y_true = y_true.toPandas()

In [ ]:
y_p = result.select('prediction')
y_p = y_p.toPandas()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_p)
cm

In [ ]:
cseg = ["Seg-1", "Seg-2", "Seg-3"]
cm_df = pd.DataFrame(cm, index = cseg, columns = cseg)

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(cm_df, annot=True, cmap = 'Blues', annot_kws={"size": 16})
plt.title('Wine Customers Confusion Matrix\n')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()